In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler,LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")
df = pd.read_csv('/content/wisconsin_data.csv')
df.head()

df.drop(columns=['id','Unnamed: 32'],inplace=True)
df.info()

X=df.drop(columns=['diagnosis'])
y=df['diagnosis']

scaler=MinMaxScaler()
X=scaler.fit_transform(X)

encoder=LabelEncoder()
y=encoder.fit_transform(y)

X_train , X_test , y_train , y_test = train_test_split(X,y ,test_size=0.2 ,shuffle=True, random_state=42)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

ANN=Sequential([
    Dense(16, activation='relu',input_dim=30),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])

ANN.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy'])
ANN.summary()

In [ ]:
history = ANN.fit(X_train, y_train, epochs=100, validation_split=0.2)

In [ ]:
tr_acc = history.history['accuracy']
tr_loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']
epochs = [i+1 for i in range(len(tr_acc))]
plt.figure(figsize=(20, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs, tr_loss, 'r', label='Train Loss')
plt.plot(epochs, val_loss, 'g', label='Valid Loss')
plt.title('Loss')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.subplot(1, 2, 2)
plt.plot(epochs, tr_acc, 'r', label='Train Accuracy')
plt.plot(epochs, val_acc, 'g', label='Valid Accuracy')
plt.title('Accuracy')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.tight_layout()
plt.show()

In [ ]:
y_pred=ANN.predict(X_test)
y_pred = (y_pred > 0.5)

In [ ]:
cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
print("\nTESTING RESULTS: \n===============================")
clf_report = pd.DataFrame(classification_report(y_test, y_pred, output_dict=True))
print(f"CLASSIFICATION REPORT:\n{clf_report}")

In [ ]:
ANN.save('ANN.h5')

In [ ]:
from tensorflow.keras.layers import BatchNormalization, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
import numpy as np
# 데이터 전처리
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 모델 정의
ANN2 = Sequential([
    Dense(64, activation='relu', input_dim=30, kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(32, activation='relu', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(16, activation='relu', kernel_regularizer=l2(0.01)),
    BatchNormalization(),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

In [ ]:
optimizer = Adam(learning_rate=0.001)

ANN2.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy', 'AUC']
    )

callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        mode='min'
        ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=5,
        min_lr=1e-6,
        mode='min'
        ),
    ModelCheckpoint(
        filepath='best_model.keras',
        monitor='val_loss',
        save_best_only=True,
        mode='min'
        )
    ]

In [ ]:
# 모델 학습
history = ANN2.fit(
    X_train, y_train,
    epochs=150,
    batch_size=32,
    validation_split=0.2,
    callbacks=callbacks,
    verbose=1
)
# 모델 평가
test_scores = ANN2.evaluate(X_test, y_test, verbose=0)
print("\n테스트 세트 성능:")
print(f"Loss: {test_scores[0]:.4f}")
print(f"Accuracy: {test_scores[1]:.4f}")
print(f"AUC: {test_scores[2]:.4f}")
# 모델 구조 요약
ANN2.summary()


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

model = SVC()
param_grid = {
    'gamma' : [0.0001, 0.001, 0.01, 0.1, 1],
    'C' : [0.01, 0.05, 0.5, 0.1, 1, 10, 20, 50, 100]
    }
grid_search = GridSearchCV(model, param_grid, cv=5, return_train_score=True)
grid_search.fit(X_train, y_train)

In [ ]:
print(grid_search.best_params_)

In [ ]:
print(grid_search.best_score_)
print(grid_search.best_estimator_)